In [1]:
# Interpola el delta de temperatura
# y precipitación para que sea compatible
# con la resolución de CHIRTS y CHIRPS.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

In [2]:
path = "../../../datos/CHIRTS/"
Delta = ["../../../datos/AIRCC-CLIM/",
    "../../../resultados/AIRCC-CLIM/"]
months = ["January", "February", "March", "April",
    "May", "June", "July", "August",
    "September", "October", "November", "December"]
years = [2030, 2050, 2070]
RCP = ["RCP4p5", "RCP8p5"]
vars = ["Tmax", "Tmin", "Tmean", "Pre"]
var_delta = ["temp", "prec"]
var_base = ["Temp", "Prec_perc"]

# Límites de México.
lon_min = -118.390
lon_max =  -86.660
lat_min =   14.566
lat_max =   32.737

# Si no existe la carpeta, la crea.
if not os.path.exists(Delta[1]):
    os.mkdir(Delta[1])
for i in RCP:
    if not os.path.exists(Delta[1] + i):
        os.mkdir(Delta[1] + i)
    if not os.path.exists(Delta[1] + i + "_base"):
        os.mkdir(Delta[1] + i + "_base")

# Archivo base para interpolar.
ds = xr.open_dataset(path +
    "Tmax.1983.01.01.tif.nc" )

In [104]:
# Interpola el delta para el
# año base de cada RCP.

fpath_d_base = []
fpath_r_base = []
y = 2005

# Crea una lista con todas las direcciones
# de los datos y de los resultados.
for i in range(len(var_delta)):
    for j in RCP:
        for m_n, m in enumerate(months):
            m_n += 1

            # Datos.
            fpath_d_base.append( Delta[0] + j
                + "_base/" + var_base[i] +
                "_SchneiderThompson_Stochastic_"
                + j + "_" + m + "_"
                + str(y) + ".nc" )

            # Resultados.
            fpath_r_base.append( Delta[1] + j
            + "_base/" + var_delta[i] +
            "_tol_Stochastic_" + j + "_" +
            f"{m_n:02d}" + "_"
            + str(y) + ".nc" )

# Iteramos para cada archivo.
for i in range(len(fpath_d_base)):
    # Abrimos el archivo y lo georreferenciamos.
    d = xr.open_dataset( fpath_d_base[i] )
    d = d.rio.set_spatial_dims("lon", "lat")
    d = d.rio.write_crs("EPSG:4326")
    
    # Interpolamos de acuerdo con CHIRTS.
    d = d.rio.clip_box( -180,
    -60, 180, 70 )
    d = d.interp_like(ds)
    
    # Recortamos la caja que delimita a México.
    d = d.rio.clip_box( lon_min,
        lat_min, lon_max, lat_max )
    d = d.drop_vars("spatial_ref")
    
    # Usamos np.nan como dato faltante.
    d = d.where(d > -9000)
    
    # Renombramos las variables del Dataset.
    if fpath_d_base[i][38:42] == var_base[0]: n = 0
    else: n = 1
    d = d.rename({var_base[n]: var_delta[n]})
    
    # Guardamos el archivo.
    d.to_netcdf( fpath_r_base[i] )

In [106]:
# Interpola el delta para los
# años futuros de cada RCP.

fpath_d = []
fpath_r = []

# Crea una lista con todas las direcciones
# de los datos y de los resultados.
for v in var_delta:
    for j in RCP:
        for y in years:
            for m_n, m in enumerate(months):
                m_n += 1

                # Datos.
                fpath_d.append( Delta[0] + j +
                    "/" + v + "_tol_Stochastic_"
                    + j + "_" + m + "_"
                    + str(y) + ".nc" )

                # Resultados.
                fpath_r.append( Delta[1] + j +
                    "/" + v + "_tol_Stochastic_"
                    + j + "_" + f"{m_n:02d}"
                    + "_" + str(y) + ".nc" )

for i in range(len(fpath_d)):
    # Abrimos el archivo y lo georreferenciamos.
    d = xr.open_dataset( fpath_d[i] )
    d = d.rio.set_spatial_dims("lon", "lat")
    d = d.rio.write_crs("EPSG:4326")
    
    # Interpolamos de acuerdo con CHIRTS.
    d = d.rio.clip_box( -180,
    -60, 180, 70 )
    d = d.interp_like(ds)
    
    # Recortamos la caja que delimita a México.
    d = d.rio.clip_box( lon_min,
        lat_min, lon_max, lat_max )
    d = d.drop_vars("spatial_ref")
    
    # Usamos np.nan como dato faltante.
    d = d.where(d > -9000)
    
    # Quizás sería más eficiente abrir todos los
    # archivos base, concatenarlos y acceder a
    # cada mes del Dataset, pero entonces se
    # dificulta trabajar con una lista de la
    # dirección del archivo y habría que tener
    # ciclos anidados con RCP, var_delta, y el mes,
    # lo que a su vez restaría eficiencia.
    b = xr.open_dataset( fpath_r[i][:37]
        + "_base" + fpath_r[i][37:-7]
        + "2005.nc" )
    
    # Quitamos el efecto climático ya
    # presente en la climatología 1995-2014.
    if fpath_r[i][38:42] == var_delta[0]:
        d -= b
    else: 
        d = ( 1 + d / 100 ) / ( 1 + b / 100 ) - 1
    
    # Guardamos el archivo.
    d.to_netcdf( fpath_r[i] )

In [7]:
# Interpola los datos de los SSP.

# Datos y directorios.
SSP = "../../../resultados/SSP/"
path_vars = "../../../resultados/vars/"
esc = ["SSP2", "SSP5"]
var_soc = ["GDP", "POP"]
var_soc_2 = ["PIB", "Poblacion"]

# Si no existe la carpeta, la crea.
for r in RCP:
    for v in var_soc_2:
        if not os.path.exists(
            path_vars + r + "/" + v ):
            os.mkdir( path_vars
                + r + "/" + v )

# Ajustamos el archivo base a las
# dimensiones de México.
ds_crop = ds.copy()
ds_crop = ds_crop.rio.write_crs("EPSG:4326")
ds_crop = ds_crop.rio.clip_box( lon_min,
    lat_min, lon_max, lat_max )
ds_crop = ds_crop.drop_vars("crs")

# Iteramos para cada escenario.
for e in range(len(esc)):

    # Nombres del archivos para el RCP.
    fname = [var_soc[0]  + "_" + esc[e] + ".nc",
        var_soc[1]  + "_" + esc[e] + ".nc"]

    # Iteramos para cada variable-
    for i in range(len(var_soc)):
        d = xr.open_dataset(SSP + fname[i])
        d = d.assign_coords( {
            "time": pd.date_range("2008-01-01", 
                "2099-01-01", freq = "1Y"),
            "lon": np.arange(0.25, 360.25, 0.5),
            "lat": np.arange(-90, 90, 0.5),
        } )

        # Iteramos para cada año.
        for f in [2010] + years:
            # Escogemos el año para la variable.
            d_year = d.isel( time = f - 2008 )

            # Adecuamos el formato del Dataset.
            d_year = d_year.drop("time")
            d_year = ( d_year.rio
                .set_spatial_dims("lon", "lat") )
            d_year = d_year.rio.write_crs("EPSG:4326")

            # Interpolamos de acuerdo con CHIRTS
            d_year = d_year.rio.clip_box( 0,
                -60, 360, 70 )
            d_year = d_year.fillna(0)
            d_year = d_year.interp_like(ds) / 100

            # Recortamos la caja que delimita a México.
            d_year = d_year.rio.clip_box( lon_min + 180,
                lat_min, lon_max + 180, lat_max )
            d_year["lon"] = ds_crop["lon"]
            d_year = d_year.drop_vars("spatial_ref")

            # Usamos np.nan como dato faltante.
            d_year = d_year.where(d_year > -9000)

            # Usamos el nombre estándar de la variable.
            d_year = d_year.rename( { var_soc[i]:
                var_soc_2[i] } )

            # Guardamos el archivo.
            d_year.to_netcdf( path_vars +
                RCP[e] + "/" + var_soc_2[i]
                + "/mexico_" + var_soc_2[i] 
                + "." + str(f) + "_yearly.nc" )